In [1]:
import numpy as np
import pandas as pd
from sympy import isprime
import matplotlib.pyplot as plt
%matplotlib inline
import threading
from itertools import permutations
import time

In [2]:
cities = pd.read_csv('../input/cities.csv')
cities.describe()

,CityId,X,Y
count,197769.000000,197769.000000,197769.000000
mean,98884.000000,2741.169734,1859.277703
std,57091.137031,1474.870239,985.144313
min,0.000000,1.871925,0.000000
25%,49442.000000,1439.292001,1054.234324
50%,98884.000000,2772.522934,1946.624393
75%,148326.000000,4246.994405,2736.536831
max,197768.000000,5099.502142,3397.809824


In [36]:
path = pd.read_csv('../input/submission_20181210.csv')
#path = pd.read_csv('../input/sample_submission.csv')
path = path['Path'].values
path

array([     0,  78934, 111804, ...,  40230,  48816,      0], dtype=int64)

In [4]:
cities_len = len(cities)
print("len: ", cities_len)
cities_xy = np.stack((cities.X.values, cities.Y.values), axis=1)
cities_xy[0:10]

len:  197769


array([[ 316.83673906, 2202.34070734],
       [4377.40597217,  336.60208217],
       [3454.15819771, 2820.05301125],
       [4688.09929763, 2935.89805581],
       [1010.69695175, 3236.75098903],
       [2474.23087706, 1435.51465103],
       [1029.2777954 , 2721.80095182],
       [3408.8876851 ,  199.58579311],
       [1304.00612487, 2657.42724641],
       [4211.5257252 , 2294.59520756]])

In [5]:
def get_score(path):
    xy = cities_xy[path]
    values = np.linalg.norm(xy-np.roll(xy, -1, axis=0), axis=1)
    prime_value = (values[9::10] * [int(isprime(i)==False)*0.1 for i in path[9::10]]).sum()
    return values.sum() + prime_value

In [37]:
get_score(path)

1524462.5023936473

In [ ]:
# def fix_path_small_permutation(seq_len:int=6):
#     start_index = np.random.choice(cities_len-seq_len)+1
#     remainder = start_index%10
#     if remainder == 0:
#         score_start_index = start_index - 10
#     else:
#         score_start_index = start_index - remainder
#     small = path[start_index : start_index + seq_len]
#     small_start = path[score_start_index:start_index]
#     small_end = path[start_index + seq_len:start_index + seq_len+1]
#     small_score = np.concatenate((small_start, small, small_end))
#     score = get_score(small_score)
#     for item in permutations(small):
#         small_path_test = np.concatenate((small_start, item, small_end))
#         score_test = get_score(small_path_test)
#         if score_test < score:
#             path[start_index : start_index + seq_len] = item
#             #print("start_index: ", start_index, "score : ", get_score(path))
#             score = score_test

In [43]:
def fix_path_small_permutation(seq_len:int=6, band_len:int=1, start_index:int=None):
    if band_len is None:
        band_len = np.random.choice(int((cities_len-1)/seq_len)) + 1
    small_len = band_len*seq_len
    if start_index is None:
        start_index = np.random.choice(cities_len - 1 - small_len)+1
    remainder = start_index%10
    if remainder == 0:
        score_start_index = start_index - 10
    else:
        score_start_index = start_index - remainder
    small = path[start_index : start_index + small_len]
    small_start = path[score_start_index:start_index]
    small_end = path[start_index + small_len:start_index + small_len+1]
    
    small_score = path[score_start_index: start_index + small_len+1]
    score = get_score(small_score)
    if band_len > 1:
        small = small.reshape((-1,band_len))
    for item in permutations(small):
        if band_len > 1:
            item = np.concatenate(item)
        small_path_test = np.concatenate((small_start, item, small_end))
        score_test = get_score(small_path_test)
        if score_test < score:
            path[start_index : start_index + small_len] = item
            print("path updated, score : ", get_score(path))
            score = score_test

In [46]:
def fix_path_use_thread(band_len:int=None, first_index:int=None):
    seq_len = 6
    if band_len is None:
        band_len = np.random.choice(int((cities_len-1)/seq_len)) + 1
    small_len = band_len * seq_len
    if first_index is None:
        first_index = np.random.choice(small_len)
    print('band_len:', band_len, ", first_index:", first_index)    
    threads_len = int((cities_len-1)/small_len)
    threads = []
    for i in range(threads_len+2):
        if i == 0:
            start_index = 1    
        else :
            start_index = first_index + (small_len * (i-1)) + 1
        
        if start_index > cities_len:
            break
        elif start_index + small_len + 1 > cities_len:
            start_index = first_index + cities_len-small_len-1
        t = threading.Thread(target=fix_path_small_permutation, args=(seq_len, band_len, start_index))
        threads.append(t)
    for t in threads:
        t.start()
    for t in threads:
        t.join()

In [47]:
print("start score : ", get_score(path))
for i in range(100):
    fix_path_use_thread()
    #print(i)
print("final score : ", get_score(path))

start score :  1524455.6944955632
band_len: 17095 , first_index: 90052


Exception in thread Thread-203289:
Traceback (most recent call last):
  File "C:\Users\southman\Anaconda3\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\Users\southman\Anaconda3\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-43-2dda0dff44b6>", line 19, in fix_path_small_permutation
    small = small.reshape((-1,band_len))
ValueError: cannot reshape array of size 12520 into shape (17095)



KeyboardInterrupt: 

In [28]:
len(path)

197770

In [ ]:
values = get_dist_values(path)
values.mean()


In [ ]:
get_score(path)

In [ ]:
# path = np.append(np.arange(cities_len), [0])
# best_path = None
# best_score = get_score(path)
# for i in range(cities_len):
#     path = permutation(path)
#     score = get_score(path)
#     if score < best_score :
#         best_score = score
#         best_path = path
# print(best_score, best_path)    

In [ ]:
path = np.append(np.arange(cities_len), [0])
shuffle(path)
best_path = None
best_score = get_score(path)

In [ ]:
down_count = 0
max_down_count = 100

while True:
    shuffle_partially(path)
    score = get_score(path)
    if score < best_score :
        best_score = score
        print(best_score)
        best_path = np.copy(path)
        down_count = 0
    else:
        down_count += 1
        if down_count > max_down_count:
            break
print(best_score, best_path)    

In [ ]:
def get_dist_values(path:np.ndarray):
    xy = cities_xy[path]
    values = np.linalg.norm(xy-np.roll(xy, -1, axis=0), axis=1)
    values[9::10] += values[9::10] * [int(isprime(i)==False)*0.1 for i in path[9::10]]
    return values

In [ ]:
far_values = values > values.mean()
path_far = path[1:][far_values[:-1]]
path_far
#np.random.shuffle(path_far)
#values[far] = path_far
#a

In [42]:
pd.DataFrame({'Path': path}).to_csv('../input/submission_20181210.csv', index=False)

In [ ]:
plt.figure(figsize=(32,18))
plt.rcParams['agg.path.chunksize'] = 10000 
plt.plot(cities.X[path], cities.Y[path])
plt.show()